In [20]:
import requests
import json
from datetime import datetime, timedelta

# Create timestamps
now = datetime.now()
first_event_time = now - timedelta(minutes=5)  # 5 minutes ago
last_event_time = now

# Build test incident payload
test_incident = {
    "id": f"TEST_{int(now.timestamp())}",
    "timestamp": now.isoformat(),
    "description": json.dumps({
        "rule_id": "9999",
        "rule_description": " Hello Erika",
        "rule_level": 10,
        "total_events": 2,
        "agent_names": ["Blue Iris"],
        "first_event_timestamp": first_event_time.isoformat(),
        "last_event_timestamp": last_event_time.isoformat(),
        "sample_event": {
            "rule": {
                "id": "834613",
                "description": "Critical Camera Alert",
                "level": 10
            },
            "agent": {
                "name": "Blue Iris",
                "id": "001"
            },
            "timestamp": first_event_time.isoformat(),
        },
        "csv_path": "/incidents/test/events.csv"
    }),
    "acknowledged": False,
    "escalated": False
}

# Send POST request using requests module
response = requests.post(
    url='http://192.168.178.40:8000/incidents/',
    headers={'Content-Type': 'application/json'},
    json=test_incident
)

if response.status_code != 200:
    print('Error:', response.status_code, response.text)
else:
    print('Response:', response.json())

Response: {'message': 'Incident created', 'id': 'TEST_1732207974'}


In [5]:
#!/usr/bin/env python3
from auth import register_user, SessionLocal, Base, engine
from fastapi import HTTPException
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def init_db():
    # Create tables
    Base.metadata.create_all(bind=engine)
    
    success = True
    try:
        # Create admin user
        admin_user = register_user("admin", "admin", role="admin")
        logger.info("Admin user created successfully")
        
        # Create analyst user
        analyst_user = register_user("analyst", "analyst", role="analyst")
        logger.info("Analyst user created successfully")
        
    except HTTPException as e:
        if e.detail == "Username already registered":
            logger.info("User already exists")
        else:
            logger.error(f"Error creating user: {e.detail}")
        success = False
    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}")
        success = False
    
    return success

if __name__ == "__main__":
    init_db()

2024-12-12 19:01:44,240 - passlib.handlers.bcrypt - DEBUG - detected 'bcrypt' backend, version '3.2.0'
2024-12-12 19:01:44,242 - passlib.handlers.bcrypt - DEBUG - 'bcrypt' backend lacks $2$ support, enabling workaround
2024-12-12 19:01:44,499 - __main__ - INFO - Admin user created successfully
2024-12-12 19:01:44,718 - __main__ - INFO - Analyst user created successfully


In [6]:
#!/usr/bin/env python3
from auth import SessionLocal, User
import logging
from sqlalchemy import inspect

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def print_all_users():
    db = SessionLocal()
    try:
        # Get all columns from User model
        inspector = inspect(User)
        columns = [c.key for c in inspector.mapper.column_attrs]
        
        # Query all users
        users = db.query(User).all()
        
        print("\nAll users in database:")
        for user in users:
            print("-" * 60)
            # Print all columns for each user
            for col in columns:
                value = getattr(user, col)
                print(f"{col}: {value}")
        print("-" * 60)
            
    except Exception as e:
        logger.error(f"Error querying database: {str(e)}")
    finally:
        db.close()

if __name__ == "__main__":
    print_all_users()


All users in database:
------------------------------------------------------------
id: 1
username: admin
password_hash: $2b$12$S5hmQetzgclZTvG6M2TTKeoHUqK09LRcry3Z4M5sTuXxZfmi1r/JW
role: admin
------------------------------------------------------------
id: 2
username: analyst
password_hash: $2b$12$0XRONatrWieQZ8X3QIwKO.9XikLJzDTTvuWWG9SaZcTzfYQ3Sm.N.
role: analyst
------------------------------------------------------------


In [5]:
enabled = "False"
if enabled == "True":
    print("Enabled")

True


In [8]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime
import bcrypt
import json
import os

SQLALCHEMY_DATABASE_URL = "sqlite:///./users.db"
engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True)
    username = Column(String(100), unique=True)  # Will use email as username
    name = Column(String(100))
    email = Column(String(100), unique=True)
    phone = Column(String(20))
    department = Column(String(50))
    role = Column(String(20))
    password_hash = Column(String(100))
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow)

def hash_password(password):
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode()

def create_initial_users():
    try:
        # Drop and recreate tables
        Base.metadata.drop_all(bind=engine)
        Base.metadata.create_all(bind=engine)
        
        db = SessionLocal()

        # Initial users with email as username
        users = [
            User(
                username="fabianannomerz@gmail.com",  # Email as username
                name="Fabian Merz",
                email="fabianannomerz@gmail.com",
                phone="**621",
                department="Security",
                role="admin",
                password_hash=hash_password("admin")
            ),
            User(
                username="merzfabi@hs-albsig.de",  # Email as username
                name="Fabian Merz Student", 
                email="merzfabi@hs-albsig.de",
                phone="**622",
                department="IT",
                role="analyst",
                password_hash=hash_password("analyst")
            )
        ]

        for user in users:
            db.add(user)
            print(f"Creating user: {user.name} (username: {user.username})")

        db.commit()
        print(f"\nSuccessfully created {len(users)} users")

    except Exception as e:
        print(f"Error: {str(e)}")
        db.rollback()
        raise
    finally:
        db.close()

if __name__ == "__main__":
    create_initial_users()

Creating user: Fabian Merz (username: fabianannomerz@gmail.com)
Creating user: Fabian Merz Student (username: merzfabi@hs-albsig.de)

Successfully created 2 users


In [9]:
#new migration
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime
import bcrypt
import os

# Database setup
SQLALCHEMY_DATABASE_URL = "sqlite:///./users.db"
engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True)
    username = Column(String(100), unique=True)
    name = Column(String(100))
    email = Column(String(100), unique=True)
    phone = Column(String(20))
    department = Column(String(50))
    role = Column(String(20))
    password_hash = Column(String(100))
    reset_token = Column(String(100), nullable=True)
    token_expiry = Column(DateTime, nullable=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow)

def hash_password(password):
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode()

def create_initial_users():
    try:
        # Remove old database
        if os.path.exists("users.db"):
            os.remove("users.db")
            print("Removed old database")

        # Create new tables
        Base.metadata.create_all(bind=engine)
        print("Created new database schema")
        
        db = SessionLocal()

        # Initial users
        users = [
            User(
                username="fabianannomerz@gmail.com",
                name="Fabian Merz",
                email="fabianannomerz@gmail.com", 
                phone="**621",
                department="Security",
                role="admin",
                password_hash=hash_password("admin")
            ),
            User(
                username="merzfabi@hs-albsig.de",
                name="Fabian Merz Student",
                email="merzfabi@hs-albsig.de",
                phone="**622", 
                department="IT",
                role="analyst",
                password_hash=hash_password("analyst")
            )
        ]

        for user in users:
            db.add(user)
            print(f"Adding user: {user.name}")

        db.commit()
        print("\nMigration completed successfully")

    except Exception as e:
        print(f"Error during migration: {str(e)}")
        db.rollback()
    finally:
        db.close()

if __name__ == "__main__":
    create_initial_users()

Removed old database
Created new database schema
Adding user: Fabian Merz
Adding user: Fabian Merz Student

Migration completed successfully
